# Content Creator Multi-Agent Workflow

## 1. Configurations

In [ ]:
!pip install -q -U "deepchecks-llm-client[otel]" boto3 botocore
!pip install -q crewai==0.203.0 crewai-tools==0.76.0
!pip install -q litellm==1.79.1

### AWS Configuration

In [ ]:
# Only run for sagemaker deplyment
import boto3
import os
sts = boto3.client("sts")
os.environ["AWS_PARTNER_APP_AUTH"] = "True"
os.environ["AWS_PARTNER_APP_ARN"] = "your-aws-partner-app-arn-key"

In [ ]:
# **Otherwise**
# Configure the AWS connection directly for bedrock usage
# Can replace with any other LLM provider
import os

os.environ["AWS_ACCESS_KEY_ID"]=
os.environ["AWS_SECRET_ACCESS_KEY"]=
os.environ["AWS_SESSION_TOKEN"]=

### Deepchecks Configuration

In [ ]:
DEEPCHECKS_HOST = "your-deepchecks-app-url-here"
DEEPCHECKS_LLM_API_KEY = "your-deepchecks-api-key-here"
DEEPCHECKS_APP_NAME = "Content Creator Crew"

In [ ]:
# Create Your Application On Deepchecks
from deepchecks_llm_client.client import DeepchecksLLMClient
from deepchecks_llm_client.data_types import ApplicationType

dc_client = DeepchecksLLMClient(api_token=DEEPCHECKS_LLM_API_KEY, host=DEEPCHECKS_HOST)
dc_client.create_application(DEEPCHECKS_APP_NAME, app_type=ApplicationType.AGENT)

### Demo Specific Configuration

In [ ]:
import pandas as pd
import os

from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import SerperDevTool
from crewai.tools import tool
from deepchecks_llm_client.otel import CrewaiIntegration
from deepchecks_llm_client.data_types import EnvType

os.environ["SERPER_API_KEY"] = "your-serper-api-key-here"

## 2. Crew Setup

In [ ]:
def create_editor_tools(llm):
    """Create editor tools with LLM dependency injected via closure."""

    @tool("Hook Improver")
    def hook_fix_tool(post, issue):
        """Improves the hook/opening of a blog post."""
        sentences = [s.strip() + '.' for s in post.split('.') if s.strip()]
        current_hook = sentences[0] if sentences else ""
        prompt = f"Improve this hook: '{current_hook}'. Issue: {issue}. Return ONLY the improved hook (1-2 sentences)."
        return llm.call(prompt).strip().strip('"\'')

    @tool("Tone Adjuster")
    def tone_fix_tool(text, current_tone, target_tone, audience):
        """Adjusts tone of content to match target audience."""
        prompt = f"Rewrite for {audience}. Change tone from '{current_tone}' to '{target_tone}': {text}"
        return llm.call(prompt).strip()

    @tool("Content Rewriter")
    def rewrite_article_tool(original_post, issues):
        """Rewrites content to fix specific issues."""
        prompt = f"Rewrite this post (~100 - 300 words) to fix: {issues}\n\nOriginal: {original_post}"
        return llm.call(prompt).strip()

    return [hook_fix_tool, tone_fix_tool, rewrite_article_tool]

print("✅ Editor Tools Ready")

In [ ]:
def create_crew(model):
    """
    Create a reusable crew with the specified model.
    Tasks use placeholders {topic}, {context}, {audience} for dynamic inputs.

    Args:
        model: Bedrock model ID (e.g., "bedrock/us.anthropic.claude-3-5-sonnet-20241022-v2:0")

    Returns:
        Crew instance ready to process inputs with placeholders
    """
    # Set up LLM and tools
    llm = LLM(model=model)
    editor_tools = create_editor_tools(llm)

    # Create agents
    writer = Agent(
        role="Content Writer",
        goal="Write engaging blog posts (100-300 words) based on topic, context, and audience.",
        backstory="""Experienced content writer who knows when to research and when to write.
        Use SerperDevTool only if you genuinely need recent data or specific facts.""",
        tools=[SerperDevTool()],
        llm=llm
    )

    reviewer = Agent(
        role="Content Reviewer",
        goal="Review posts for quality, tone, engagement, and substance.",
        backstory="""Senior editor who maintains quality standards without being overly harsh.
        You notice when facts or statistics feel off or need verification.
        Good enough is often good enough - focus on meaningful issues.""",
        tools=[SerperDevTool()],
        llm=llm
    )

    editor = Agent(
        role="Content Editor",
        goal="Improve posts based on reviewer feedback.",
        backstory="""You're a perfectionist editor with an eye for excellence.
        You believe good writing can always become great writing.
        You're curious about facts and claims - when something sounds off, you wonder if it's accurate.
        When you see a way to enhance something, you act on it.
        You're not satisfied with 'good enough' - you're driven to make every piece the best it can be.""",
        tools=editor_tools + [SerperDevTool()],
        llm=llm
    )

    # Define tasks with placeholders for dynamic inputs
    writing_task = Task(
        description="""Write a ~100-300 word blog post about: {topic}
        Context: {context}
        Audience: {audience}
        Return ONLY the blog post text (no JSON, no metadata).""",
        agent=writer,
        expected_output="The blog post as plain text"
    )

    review_task = Task(
        description="""Review the post about "{topic}" for audience: {audience}
        Check: tone, hook quality, substance, engagement.
        Return JSON with: approved (bool), feedback_summary (string), improvement_suggestions (list).""",
        agent=reviewer,
        context=[writing_task],
        expected_output="JSON with approval status and feedback"
    )

    editing_task = Task(
        description="""Review the post and apply your editorial judgment.
        Topic: {topic} | Audience: {audience}
        Return ONLY the final post text (no JSON, no metadata).""",
        agent=editor,
        context=[writing_task, review_task],
        expected_output="The final blog post as plain text"
    )

    # Create and return crew
    return Crew(
        agents=[writer, reviewer, editor],
        tasks=[writing_task, review_task, editing_task],
        process=Process.sequential,
        verbose=False
    )

print("✅ Crew Ready")

### Utils

In [ ]:
def start_deepchecks_tracing(version_name, tracer_provider=None):
    """Set up Deepchecks tracing for a specific version.

    Args:
        version_name: Name of the version for tracking
        tracer_provider: Previous tracer to uninstrument (optional)

    Returns:
        tracer_provider: New tracer provider instance
    """
    # Uninstrument previous tracer if it exists
    if tracer_provider:
        CrewaiIntegration().uninstrument()

    # Create new tracing location
    tracer_provider = CrewaiIntegration().register_dc_exporter(
        host=DEEPCHECKS_HOST,
        api_key=DEEPCHECKS_LLM_API_KEY,
        app_name=DEEPCHECKS_APP_NAME,
        version_name=version_name,
        env_type=EnvType.EVAL,
    )
    print(f"✅ Tracing: {DEEPCHECKS_APP_NAME} / {version_name}\n")
    return tracer_provider

In [ ]:
def run_batch(model, df):
    """
    Run content creation workflow for all rows in a dataframe.

    Args:
        model: Bedrock model ID to use for content generation
        df: DataFrame with columns 'topic', 'audience', and optional 'context'

    Returns:
        List of CrewOutput objects with results for each input
    """
    results = []
    for idx, row in df.iterrows():
        topic = str(row.get('topic', '')).strip()
        context = str(row.get('context', '')).strip()
        audience = str(row.get('audience', '')).strip()

        if not topic or not audience:
            continue

        try:

            crew = create_crew(model)
            result = crew.kickoff(inputs={
                'topic': topic,
                'context': context,
                'audience': audience
            })
            results.append({'row': idx+1, 'topic': topic, 'result': result})
        except Exception as e:
            print(f"❌ Error on row {idx+1}: {e}")

    print(f"\n✅ Batch complete: {len(results)}/{len(df)} successful")
    return results

print("✅ Workflow Ready")

### Test Single Input

In [ ]:
# First let's run one input to test the setup
model = "bedrock/us.anthropic.claude-3-5-sonnet-20241022-v2:0"
topic = "Why 'just build stuff' is the best advice for new developers."
audience = "Junior software developers and coding bootcamp students."
context = ""

crew = create_crew(model)
result = crew.kickoff(inputs={'topic': topic, 'audience': audience, 'context': context})
print(result)

## 3. Baseline - Evaluate The Crew

In [ ]:
# Load Data

df = pd.read_csv('https://ndownloader.figshare.com/files/61017985')
df.head()

In [ ]:
# Set up tracing
tracer_provider = start_deepchecks_tracing("Baseline - Claude 3.5 Sonnet")

# Run all of the inputs
results_v1_claude_3_5_sonnet = run_batch(model, df)

> Note: Go to the Deepchecks platform to see the results before proceeding to the next step



## 4. Enhanced - Better Descriptions For The Tools

In our previous version we didn't write a clear description of what goes into the tool, and we didn't have type hints for the functions, because of that our editor failed calling the tools.

In [ ]:
def create_editor_tools(llm):
    """Create editor tools with improved descriptions and type hints."""

    @tool("Hook Improver")
    def hook_fix_tool(post: str, issue: str) -> str:
        """Improves the hook/opening of a blog post. Returns improved hook text.

        Args:
            post: The full blog post text
            issue: What's wrong with the hook (e.g., 'too clickbaity', 'too weak')
        """
        sentences = [s.strip() + '.' for s in post.split('.') if s.strip()]
        current_hook = sentences[0] if sentences else ""

        prompt = f"Improve this hook: '{current_hook}'. Issue: {issue}. Return ONLY the improved hook (1-2 sentences)."
        response = llm.call(prompt)
        return response.strip().strip('"\'')

    @tool("Tone Adjuster")
    def tone_fix_tool(text: str, current_tone: str, target_tone: str, audience: str) -> str:
        """Adjusts tone of content to match target audience. Returns rewritten text.

        Args:
            text: Content to adjust
            current_tone: Current tone (e.g., 'too formal')
            target_tone: Desired tone (e.g., 'conversational')
            audience: Target audience description
        """
        prompt = f"Rewrite for {audience}. Change tone from '{current_tone}' to '{target_tone}': {text}"
        response = llm.call(prompt)
        return response.strip()

    @tool("Content Rewriter")
    def rewrite_article_tool(original_post: str, issues: str) -> str:
        """Rewrites content to fix specific issues. Returns complete rewritten post.

        Args:
            original_post: Current post content
            issues: Problems to fix
        """
        prompt = f"Rewrite this post (~100 - 300 words) to fix: {issues}\n\nOriginal: {original_post}"
        response = llm.call(prompt)
        return response.strip()

    return [hook_fix_tool, tone_fix_tool, rewrite_article_tool]

print("✅ Enhanced - Editor Tools Ready")

## 5. Evaluate The Crew With Better Tool Descriptions

In [ ]:
# Same model as before

# Set up tracing for the second version
tracer_provider = start_deepchecks_tracing("Enhanced - Claude 3.5 Sonnet", tracer_provider)

# Run all of the inputs
results_v2_claude_3_5_sonnet = run_batch(model, df)

## 6. Evaluate the Crew with Different Models

### Nova Micro

In [ ]:
model = "bedrock/us.amazon.nova-micro-v1:0"

tracer_provider = start_deepchecks_tracing("Nova Micro", tracer_provider)

results_nova_micro = run_batch(model, df)

### Claude 3.7 Sonnet

In [ ]:
model = "bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0"

tracer_provider = start_deepchecks_tracing("Claude 3.7 Sonnet", tracer_provider)

results_claude_3_7_sonnet = run_batch(model, df)